# Crear agentes de IA para recomendaciones de e-commerce con ADK + Vector Search

Autor(es) original(es): [Kaz Sato](https://github.com/kazunori279)

En este tutorial, exploraremos cómo construir un sistema multi-agente simple para un sitio de comercio electrónico, diseñado para ofrecer las **“Recomendaciones Generativas”** que aparecen en la demo de [Shopper's Concierge](https://www.youtube.com/watch?v=LwHPYyw7u6U). También está disponible una guía paso a paso en video [aquí](https://youtu.be/07GX28rk7Yc).

### ¿Qué es la Recomendación Generativa?

La Recomendación Generativa se refiere a la capacidad de la IA no solo de recuperar artículos que coinciden directamente con la consulta explícita del usuario, sino también de **inferir, expandir o crear inteligentemente nuevas consultas de búsqueda y sugerencias de productos basadas en una comprensión más profunda de la intención del usuario, investigación externa o información contextual.**

Aquí tienes un desglose de sus capacidades, tal como se demuestran:

1. **Comprender y expandir la intención del usuario:** En lugar de simplemente tomar una consulta directa, el sistema puede interpretar la necesidad subyacente. Por ejemplo, cuando un usuario pide un “regalo de cumpleaños para un niño de 10 años”, la IA no solo busca esas palabras clave exactas.
2. **Aprovechar la investigación externa (Google Search):** El proceso de “Recomendación Generativa” implica usar herramientas como Google Search para realizar investigación de mercado. Esto permite que la IA comprenda qué tipo de artículos suelen comprarse para una intención determinada (por ejemplo, regalos populares para niños de 10 años).
3. **Generar nuevas consultas:** Con base en esta investigación, la IA *genera* una lista de consultas más específicas y diversas. Estas consultas generadas van más allá de la entrada inicial del usuario, buscando ampliar la exploración y encontrar resultados más relevantes (por ejemplo, “juguetes educativos para niños de 10 años”, “libros de aventuras para niños”, “kits de programación para niños”).

En esencia, la “Recomendación Generativa” va más allá del simple emparejamiento de palabras clave hacia un enfoque más dinámico e inteligente, donde la IA ayuda proactivamente al usuario generando nuevas ideas y rutas de búsqueda relevantes para ayudarle a descubrir productos deseados.


In [1]:
#@title ## Instalar ADK
#@markdown Primero, instalaremos el ADK. En Colab Enterprise, puede aparecer el mensaje `ERROR: pip's dependency resolver does...`, pero puedes ignorarlo.
#@markdown ---
%pip install --upgrade google-adk==1.4.2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 5.8 MB/s eta 0:00:00


In [2]:
#@title ### Import the libraries

# Import necessary libraries
import os
import logging
import asyncio
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import  InMemorySessionService
from google.adk.tools.agent_tool import AgentTool
from google.genai import types

# Ignore warnings from ADK and Gemini APIs
logging.getLogger("google.adk.runners").setLevel(logging.ERROR)
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [3]:
#@title ### Configurar variables de entorno
#@markdown Obtén una clave de API desde Google AI Studio, colócala en GOOGLE_API_KEY a continuación y ejecuta la celda para establecer las variables de entorno necesarias para ejecutar ADK.

from getpass import getpass

# Set environment variables required for running ADK (with Gemini API Key)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
os.environ["GOOGLE_API_KEY"] = getpass("Enter your Gemini API Key: ")

# To use Vertex AI instead of Gemini API Key in Colab Enterprise or Cloud Workbench, use the following:
#[PROJECT_ID] = !gcloud config list --format "value(core.project)"
#os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
#os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
#os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

Enter your Gemini API Key: ··········


In [4]:
#@title ### Definir la función test_agent para probar agentes
#@markdown Para probar los agentes que construiremos, necesitamos definir una función test_agent que utilice Runner y SessionService para emular un entorno de ejecución de agentes. Para obtener más información sobre el entorno de ejecución de agentes, consulta la documentación de Agent Runtime (https://google.github.io/adk-docs/runtime/)


from google.genai import types

# Define the app_name, user_id and session_id for testing the agents
APP_NAME = "shop_concierge_app"
USER_ID = "user_1"

session_service = InMemorySessionService()

async def test_agent(query, agent):
  """Sends a query to the agent and prints the final response."""

  print(f"\n>>> User Query: {query}")

  # Create a session
  session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
  )

  # Create a Runner
  runner = Runner(
      app_name=APP_NAME,
      agent=agent,
      session_service=session_service,
  )

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = None
  # We iterate through events from run_async to find the final answer.
  async for event in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=content):
      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          break
  print(f"<<< Agent Response: {final_response_text}")

In [5]:
#@title ### Definir un agente de tienda
#@markdown Vamos a definir un agente de tienda y probarlo. Ten en cuenta que este agente no tiene ninguna capacidad de búsqueda por el momento.

instruction = f'''
    Tu rol es ser un agente de búsqueda de una tienda llmada Ke comercio electrónico ecuatoriano en un sitio de comercio
    electrónico con millones de artículos. Tu responsabilidad es buscar artículos
    basados en las consultas de los usuarios.
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        'Agente de tienda para un sitio de comercio electrónico llamado tienda Ke?'
    ),
    instruction=instruction,
)


In [6]:
await test_agent("¿Qué tipo de sitio es este?", shop_agent)


>>> User Query: ¿Qué tipo de sitio es este?
<<< Agent Response: Este es un sitio de comercio electrónico ecuatoriano llamado "Ke comercio electrónico ecuatoriano".


In [7]:
#@title Definir call_vector_search para llamar al backend de Búsqueda Vectorial
#@markdown Con el agente básico anterior, nos gustaría agregar la capacidad de búsqueda de ítems. Para lograr esto, aquí definimos una función `call_vector_search` que envía una solicitud HTTP a un endpoint REST proporcionado por la demo interactiva de Búsqueda Vectorial. Para conocer los detalles de cada parámetro enviado al endpoint, consulta la página de la demo.

import requests
import json

def call_vector_search(url, query, rows=None):
    """
    Llama al backend de Búsqueda Vectorial para realizar consultas.

    Args:
        url (str): La URL del endpoint de búsqueda.
        query (str): La cadena de consulta.
        rows (int, optional): Número de resultados a devolver. Por defecto es None.

    Returns:
        dict: La respuesta en formato JSON de la API.
    """

    # Construir encabezados HTTP y el payload
    headers = {'Content-Type': 'application/json'}
    payload = {
        "query": query,
        "rows": rows,
        "dataset_id": "mercari3m_mm",  # Usar el índice multimodal Mercari 3M
        "use_dense": True,              # Activar búsqueda multimodal
        "use_sparse": True,             # Activar búsqueda por palabras clave
        "rrf_alpha": 0.5,               # Mezclar resultados semánticos y por palabras clave con igual peso
        "use_rerank": True,             # Usar API de ranking para reordenar resultados
    }

    # Enviar solicitud HTTP al endpoint de búsqueda
    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()     # Lanza excepción si el código HTTP es malo
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al llamar a la API: {e}")
        return None


In [8]:
#@title Definir la herramienta **find_shopping_items**
#@markdown Ahora, envolveremos la función `call_vector_search` con una herramienta ADK llamada `find_shopping_items`. Ten en cuenta que necesitamos:
#@markdown 1) Usar la tipificación explícita, como `queries: list[str]`
#@markdown 2) Usar un docstring detallado, tanto para transmitir la funcionalidad como la semántica de esta herramienta al agente.

#@markdown Para más detalles sobre el mecanismo de herramientas de ADK, consulta la sección [Tools](https://google.github.io/adk-docs/tools/) en la documentación de ADK.

from typing import Dict

def find_shopping_items(queries: list[str]) -> Dict[str, str]:
    """
    Buscar artículos en el sitio de comercio electrónico a partir de una lista
    de consultas especificadas.

    Args:
        queries: lista de consultas a realizar.
    Returns:
        Un diccionario con las siguientes propiedades:
            - "status": estado de la ejecución:
                - "success": ejecución exitosa
            - "items": lista de artículos encontrados en el sitio de comercio electrónico.
    """
    url = "https://www.ac0.cloudadvocacyorg.joonix.net/api/query"

    items = []
    for query in queries:
        result = call_vector_search(
            url=url,
            query=query,
            rows=3,
        )
        items.extend(result["items"])

    print("-----")
    print(f"Consultas del usuario: {queries}")
    print(f"Artículos encontrados: {len(items)}")
    print("-----")

    return items


Let's test this tool.

In [9]:
find_shopping_items(["Tazas con personas bailando", "Tazas con animales bailando"])

-----
Consultas del usuario: ['Tazas con personas bailando', 'Tazas con animales bailando']
Artículos encontrados: 6
-----


[{'dense_dist': 0.0,
  'description': 'Princess house Tazas   nuevas set de 4 \nPrecio firme',
  'id': 'm91263943522',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m91263943522_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Princess house tazas',
  'rerank_score': 0.0,
  'sparse_dist': 0.3692583739757538,
  'url': 'https://www.mercari.com/us/item/m91263943522'},
 {'dense_dist': 0.20508119463920593,
  'description': 'Vintage hardanger dancer bergquist figgjo coffee mugs',
  'id': 'm10172014563',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m10172014563_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Vintage Berquist Figgjo coffee mugs',
  'rerank_score': 0.0,
  'sparse_dist': None,
  'url': 'https://www.mercari.com/us/item/m10172014563'},
 {'dense_dist': 0.19299083948135376,
  'description': 'VINTAGE CERAMIC MUG \nHOLLAND THEME \nCRAZING INSIDE NO CHIPS OR CRACKS\nBC MARKED ON BOTTOM \nSMOKE-FREE HOME\n\n\n\n\n3/23-2\n1lb 4oz\nSHD',
  'id': 'm31752226664',
  'img_

In [10]:
#@title Agregar la herramienta al **agente de compras**
#@markdown Ya está listo para agregar la herramienta al agente de compras. Se agregan las siguientes partes:

#@markdown - Adición a la `instruction`:
#@markdown `Para encontrar ítems, utiliza la herramienta find_shopping_items pasando una lista de consultas, y responde al usuario con el nombre del ítem, su descripción y img_url.`

#@markdown - Agregar el parámetro `tools` al constructor del `Agent`:
#@markdown `tools=[find_shopping_items]`

instruction = f'''
    Tu rol es ser un agente de búsqueda en un sitio de comercio electrónico con millones de
    artículos. Tu responsabilidad es buscar artículos basados en las consultas que recibas.

    Para encontrar ítems, utiliza la herramienta `find_shopping_items` pasando una lista de consultas,
    y responde al usuario con el nombre del ítem, su descripción y la img_url.
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        'Agente de tienda para un sitio de comercio electrónico'
    ),
    instruction=instruction,
    tools=[find_shopping_items],
)

Let's test the agent.

In [29]:
#@markdown Se ha alcanzado la cuota de la API
await test_agent("Tazas con figuras bailando", shop_agent)


>>> User Query: Tazas con figuras bailando


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 57.726925189s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.

In [12]:
#@title Definir el **agente de investigación** con soporte de Google Search
#@markdown A continuación, definiremos otro agente llamado `research_agent`. Este agente tomará la consulta del usuario y utilizará la herramienta integrada de Google Search para investigar qué tipo de ítems están comprando las personas según la intención del usuario. Luego generará 5 consultas para encontrar esos ítems.

#@markdown Ten en cuenta que la definición del agente siguiente especifica `google_search` como herramienta. Con esto, el agente podrá investigar tendencias antes de generar consultas de búsqueda para el sitio de e-commerce.

from google.adk.tools import google_search

instruction = f'''
    Tu rol es ser un investigador de mercado para un sitio de comercio electrónico con millones de
    artículos.

    Cuando recibas una solicitud de búsqueda de un usuario, utiliza la herramienta Google Search para
    investigar qué tipo de ítems están comprando las personas según la intención del usuario.

    Luego, genera 5 consultas para encontrar esos ítems en el sitio de comercio electrónico y
    devuélvelas.
'''

research_agent = Agent(
    model='gemini-2.5-flash',
    name='research_agent',
    description=('''
        Investigador de mercado para un sitio de comercio electrónico.
        Recibe una solicitud de búsqueda de un usuario y devuelve una lista de 5 consultas generadas en inglés.
    '''),
    instruction=instruction,
    tools=[google_search],
)


Let's test the agent.

In [13]:
await test_agent("regalo de cumpleaños para un niño de 10 años", research_agent)


>>> User Query: regalo de cumpleaños para un niño de 10 años
<<< Agent Response: Aquí tienes 5 consultas para encontrar regalos de cumpleaños para un niño de 10 años en un sitio de comercio electrónico, basadas en las tendencias de compra actuales:

1.  "Kits de robótica y ciencia para niños 10 años"
2.  "Juegos de construcción avanzados para niños 10 años"
3.  "Drones o coches teledirigidos para niños 10 años"
4.  "Juegos de mesa de estrategia para niños 10 años"
5.  "Artículos deportivos o para actividades al aire libre niño 10 años"


In [14]:
#@title Finalizar el **agente de compras**
#@markdown Para concluir, modificamos el `shop_agent` para que utilice tanto al `research_agent` como a la herramienta `find_shopping_items`.

#@markdown Probemos el agente
#@markdown Primero, el usuario le pide al agente que busque ítems. El agente de compras llamará al agente de investigación para generar consultas usando los resultados de Google Search.


instruction = f'''
    Tu rol es ser un asistente personal de compras para un sitio de comercio electrónico con millones de
    artículos. Sigue los siguientes pasos:

    Cuando recibas una solicitud de búsqueda de un usuario, pásala a la herramienta `research_agent`
    y recibe 5 consultas generadas. Luego, pasa la lista de consultas a
    `find_shopping_items` para encontrar los ítems. Cuando recibas la lista de ítems de
    la herramienta, responde al usuario con el nombre del ítem, su descripción y la URL de la imagen.
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        'Asistente personal de compras para un sitio de comercio electrónico'
    ),
    instruction=instruction,
    tools=[
        AgentTool(agent=research_agent),
        find_shopping_items,
    ],
)


In [15]:
#@markdown Nos quedamos sin la capa gratuita
await test_agent("¿Puedes encontrar un regalo de cumpleaños para un hijo de 10 años?", shop_agent)


>>> User Query: ¿Puedes encontrar un regalo de cumpleaños para un hijo de 10 años?


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}



---



## Pseudocódigo del Sistema de Google Colab (Interacción con Agentes ADK)

En este tutorial, recorrimos el proceso de construcción de un sistema de múltiples agentes para una plataforma de comercio electrónico, enfocándonos en "Recomendaciones Generativas".

A lo largo de esta progresión, el tutorial mostró cómo construir un sistema sofisticado de agentes de IA que pueda comprender la intención del usuario, realizar investigaciones, generar consultas específicas y proporcionar recomendaciones de productos relevantes en un contexto de comercio electrónico utilizando ADK y capacidades de búsqueda externas.


Este pseudocódigo describe la interacción entre el usuario, Google Colab y un sistema multi-agente construido con ADK (Agent Development Kit), como el que se ha demostrado en este cuaderno.

```pseudocode
INICIO DEL SISTEMA COLAB

  // Fase de Inicialización
  CARGAR ENTORNO_COLAB (CPU/GPU/TPU, Librerías, Variables de Entorno)
  
  // Se instala el ADK y otras dependencias
  EJECUTAR CELDA_INSTALL_ADK()
  
  // Se importan las librerías necesarias
  EJECUTAR CELDA_IMPORT_LIBRARIES()

  // Se configura la clave de API para los modelos de IA
  SOLICITAR_API_KEY()
  CONFIGURAR_API_KEY_ENV()

  // Se define una función de utilidad para probar los agentes
  DEFINIR FUNCION test_agent(consulta_usuario, agente_a_probar):
    CREAR UNA NUEVA SESION_ADK para el usuario
    INICIALIZAR RUNNER_ADK con el agente y la sesión
    ENVIAR MENSAJE_USUARIO (consulta_usuario) al RUNNER_ADK
    
    // El runner orquesta la ejecución del agente
    MIENTRAS RUNNER_ADK NO HA DADO RESPUESTA_FINAL:
      PROCESAR EVENTO_ADK:
        SI EVENTO ES LLAMADA_A_HERRAMIENTA:
          IDENTIFICAR HERRAMIENTA_REQUERIDA
          EJECUTAR HERRAMIENTA_REQUERIDA con los parámetros provistos por el agente
          DEVOLVER RESULTADO_HERRAMIENTA al agente
        
        SI EVENTO ES RESPUESTA_AGENTE_INTERMEDIA:
          MOSTRAR MENSAJE_INTERMEDIO (opcional, para depuración)

        SI EVENTO ES RESPUESTA_FINAL:
          RETORNAR RESPUESTA_FINAL.texto

  // Se define la función de búsqueda vectorial (función auxiliar)
  DEFINIR FUNCION call_vector_search(url, query, rows):
    PREPARAR SOLICITUD_HTTP con query y parámetros de búsqueda (dataset_id, use_dense, etc.)
    ENVIAR SOLICITUD_HTTP a url
    SI ERROR_HTTP:
      REGISTRAR_ERROR()
      RETORNAR_NULL()
    RETORNAR RESPUESTA_JSON

  // Se define la herramienta para buscar ítems de compras
  DEFINIR HERRAMIENTA find_shopping_items(consultas_list):
    PARA CADA consulta EN consultas_list:
      LLAMAR FUNCION call_vector_search(url_vector_search, consulta, 3)
      AGREGAR RESULTADOS.items A LISTA_ITEMS
    RETORNAR LISTA_ITEMS

  // Se define el Agente de Tienda (Versión inicial sin herramientas)
  DEFINIR AGENTE shop_agent_v1:
    NOMBRE: 'shop_agent'
    MODELO: 'gemini-2.5-flash'
    INSTRUCCIONES: 'Tu rol es ser un agente de búsqueda de una tienda...'
  
  // Se prueba el Agente de Tienda v1
  EJECUTAR test_agent("¿Qué tipo de sitio es este?", shop_agent_v1)

  // Se define el Agente de Investigación (con Google Search)
  DEFINIR AGENTE research_agent:
    NOMBRE: 'research_agent'
    MODELO: 'gemini-2.5-flash'
    INSTRUCCIONES: 'Tu rol es ser un investigador de mercado...'
    HERRAMIENTAS: [google_search] // Herramienta integrada de Google Search
  
  // Se prueba el Agente de Investigación
  EJECUTAR test_agent("regalo de cumpleaños para un niño de 10 años", research_agent)

  // Se define el Agente de Tienda (Versión final con ambas herramientas)
  DEFINIR AGENTE shop_agent_final:
    NOMBRE: 'shop_agent'
    MODELO: 'gemini-2.5-flash'
    INSTRUCCIONES: 'Tu rol es ser un asistente personal de compras...'
    HERRAMIENTAS: [
      AGENT_TOOL(research_agent), // Herramienta que llama a otro agente
      find_shopping_items         // Herramienta local para búsqueda de ítems
    ]
  
  // Se prueba el Agente de Tienda final
  EJECUTAR test_agent("¿Puedes encontrar un regalo de cumpleaños para un hijo de 10 años?", shop_agent_final)

FIN DEL SISTEMA COLAB
```

**Explicación General:**

1.  **Entorno Colab**: El sistema comienza inicializando el entorno de Google Colab, que incluye el hardware subyacente (CPU/GPU/TPU) y la carga de librerías esenciales.
2.  **Configuración ADK**: Se instala el ADK (Agent Development Kit) y se configuran las credenciales (como la clave de API de Gemini) para permitir la comunicación con los modelos de IA.
3.  **Definición de Herramientas**: Se crean funciones Python que encapsulan funcionalidades específicas (como `call_vector_search` para interactuar con un backend de búsqueda vectorial). Estas funciones se convierten en "herramientas" para que los agentes de IA puedan invocarlas.
4.  **Definición de Agentes**: Se definen los "agentes" utilizando el ADK. Cada agente tiene:
    *   Un `nombre` y una `descripción`.
    *   Un `modelo` de lenguaje grande (LLM) subyacente (ej., `gemini-2.5-flash`).
    *   `instrucciones` detalladas que guían su comportamiento y le indican cuándo y cómo usar las herramientas.
    *   Una lista de `herramientas` que puede utilizar, que pueden ser herramientas integradas (como `google_search`) o herramientas personalizadas (como `find_shopping_items`), o incluso otros agentes (como `research_agent` dentro de `shop_agent`).
5.  **`test_agent` (Ejecutor/Runner)**: Esta función simula la interacción del usuario. Cuando el usuario envía una consulta, `test_agent` utiliza el `Runner` de ADK para orquestar al agente. El `Runner` interpreta las intenciones del agente. Si el agente decide usar una herramienta, el `Runner` la ejecuta y pasa el resultado de vuelta al agente.
6.  **Flujo Multi-Agente (Ejemplo `shop_agent_final`)**:
    *   Un `shop_agent` recibe la consulta inicial del usuario.
    *   Basado en sus `instrucciones`, el `shop_agent` delega la tarea de "investigación" al `research_agent` (usándolo como una herramienta).
    *   El `research_agent` utiliza la herramienta `google_search` para obtener información externa y luego genera una lista de consultas más específicas.
    *   Estas consultas son devueltas al `shop_agent`.
    *   El `shop_agent` toma estas consultas y las pasa a la herramienta `find_shopping_items`.
    *   La herramienta `find_shopping_items` interactúa con el backend de búsqueda vectorial.
    *   Los resultados de la búsqueda (ítems) son devueltos al `shop_agent`.
    *   Finalmente, el `shop_agent` formula una respuesta amigable para el usuario con los ítems encontrados.

Este proceso ilustra cómo los agentes pueden colaborar, utilizando herramientas y delegando tareas entre sí para resolver problemas complejos de manera modular y eficiente.


# Sección 1: Orquestación de Pruebas Combinatorias (5 puntos)



Utilizar la función `generate_combinatorial_test_cases` ya definida para crear un conjunto de consultas variadas basadas en la plantilla y las configuraciones especificadas. Este es el primer paso para obtener el pool de casos de prueba. Para ello, se debe definir una plantilla de consulta base como una cadena de texto con placeholders (por ejemplo, `"regalo de {ocasion} para un {genero} de {edad} años"`), crear un diccionario llamado `variations` donde las claves sean los placeholders de la consulta base y los valores sean listas de posibles opciones para cada placeholder, luego llamar a la función `generate_combinatorial_test_cases` pasando la plantilla de consulta base y el diccionario `variations` como argumentos, y finalmente almacenar la lista de consultas generadas en una variable, por ejemplo, `combinatorial_queries`. **Nota:** Este paso ya ha sido ejecutado en la celda `d4b7fbdb` del notebook. La celda define la plantilla de consulta base, el diccionario de variaciones, llama a la función `generate_combinatorial_test_cases` y almacena los resultados en `combinatorial_queries`.


### Hallazgos Clave del Análisis de Datos

Se generó inicialmente un conjunto de 12 casos de prueba combinatorios basados en una plantilla predefinida y sus variaciones, proceso que ya se había completado en una celda previa del notebook. A continuación, se desarrolló la función `assign_risk_score` para calcular un puntaje de riesgo para cada consulta, teniendo en cuenta la presencia de palabras clave sensibles (sumando entre 3 y 5 puntos), combinaciones inusuales de parámetros (sumando 10 puntos) y la complejidad de la consulta (sumando 1 punto por cada 5 palabras). La aplicación de esta función sobre las primeras 12 consultas combinatorias resultó en puntajes uniformemente bajos, lo que indicó que no se activaron los criterios de mayor riesgo. Para demostrar la priorización de manera efectiva, se ampliaron las variaciones, incrementando el total de consultas a 36. Con las variaciones extendidas, la función `assign_risk_score` pudo diferenciar correctamente las consultas según su nivel de riesgo, con consultas que contenían palabras clave sensibles y combinaciones inusuales recibiendo puntajes más altos, mientras que las más simples obtuvieron puntajes bajos. Además, se diseñó un esquema conceptual detallado para almacenar los resultados de la ejecución de pruebas, que incluye campos clave como `test_case_id`, `query`, `initial_risk_score`, `execution_status` (éxito/fallo), `final_response`, `error_message`, `execution_time_ms`, detalles de `tool_calls` y `feedback` opcional. Se conceptualizó el uso de Machine Learning para mejorar el proceso, entrenando modelos de clasificación o regresión con datos históricos para predecir la probabilidad de fallo y ajustar dinámicamente los puntajes de riesgo, así como utilizando modelos generativos para crear nuevos casos de prueba de alto impacto. Finalmente, se implementó un bucle para iterar sobre los 36 casos de prueba priorizados, simulando la ejecución por un agente y demostrando cómo se procesarían y registrarían los resultados para análisis futuros.



In [17]:
#@title ### Genere casos de prueba combinatorios automáticamente.
#@markdown Desarrollar una función conceptual para asignar un "nivel de riesgo" a cada caso de prueba.
#@markdown "assign_risk_score" asigna un puntaje de riesgo a cada consulta según la coherencia entre edad y tipo de usuario, presencia de palabras clave sensibles y complejidad del texto, para priorizar casos de prueba críticos.

def assign_risk_score(query: str) -> int:
    """
    Asigna un nivel de riesgo a una consulta basada en su contenido.

    Args:
        query: La consulta de texto a evaluar.

    Returns:
        Un puntaje de riesgo entero.
    """
    risk_score = 0
    query_lower = query.lower()

    # 3. Lógica para palabras clave sensibles
    if "aniversario" in query_lower:
        risk_score += 5  # Incremento por palabra clave sensible
    if "sensible" in query_lower:
        risk_score += 3

    # 4. Lógica para combinaciones de parámetros inusuales o de alto riesgo
    unusual_combinations = [
        "adulto de 8 años",
        "adulto de 10 años",
        "adulto de 12 años",
        "niño de 30 años",
        "niña de 30 años"
    ]
    for combo in unusual_combinations:
        if combo in query_lower:
            risk_score += 10 # Mayor incremento por combinación inusual/alto riesgo

    # 5. Incremento basado en la longitud/complejidad de la consulta
    # Consideramos un query más largo como potencialmente más complejo o propenso a errores.
    word_count = len(query_lower.split())
    risk_score += word_count // 5 # Añade 1 punto de riesgo por cada 5 palabras

    return risk_score

# Demostración de uso de la función
import polars as pl

print("--- Demostración de assign_risk_score ---")
example_queries = [
    "regalo de cumpleaños para un niño de 10 años",
    "regalo de navidad para un niña de 8 años",
    "regalo de aniversario para un adulto de 30 años", # Contiene 'aniversario' y 'adulto de 30 años'
    "buscando un producto muy largo y detallado que podria ser complejo",
    "regalo de cumpleaños para un adulto de 8 años sensible"
]

# Recolectar datos para el DataFrame
data = []
for i, q in enumerate(example_queries):
    score = assign_risk_score(q)
    data.append({"Query": q, "Riesgo": score})

# Convertir a DataFrame de Polars
df = pl.DataFrame(data)

print("\nResultados de asignación de riesgo (Polars DataFrame):")
display(df)
print("------------------------------------------")

--- Demostración de assign_risk_score ---

Resultados de asignación de riesgo (Polars DataFrame):


Query,Riesgo
str,i64
"""regalo de cumpleaños para un n…",1
"""regalo de navidad para un niña…",1
"""regalo de aniversario para un …",6
"""buscando un producto muy largo…",2
"""regalo de cumpleaños para un a…",15


------------------------------------------


### Genere casos de prueba combinatorios automáticamente.

Función assign_risk_score asigna nivel de riesgo según coherencia edad/tipo de usuario, palabras clave sensibles y complejidad. Prioriza casos críticos: primero combinaciones inusuales, luego palabras sensibles, después complejidad, y finalmente casos estándar. Esto asegura que los escenarios con mayor probabilidad de fallo se prueben primero.

Los parámetros (edad, género, ocasión) determinan el riesgo. Bajo riesgo: combinaciones estándar coherentes (niño 8 años en cumpleaños). Medio riesgo: contienen palabras sensibles (aniversario, sensible). Alto riesgo: combinaciones inusuales (adulto 8 años) o sensibles con edades inesperadas. Estos casos deben ejecutarse primero.



In [18]:
#@title Devuelve la lista completa de casos generados

import itertools

def generate_combinatorial_test_cases(base_query_template: str, variations: dict) -> list:
    """
    Genera casos de prueba combinatorios a partir de una plantilla y variaciones.

    Args:
        base_query_template: Una cadena de texto con placeholders (ej. 'regalo de {ocasion}').
        variations: Un diccionario donde las claves son los placeholders y los valores son listas de opciones.

    Returns:
        Una lista de cadenas de texto, cada una representando un caso de prueba generado.
    """
    # Extraer las claves de los placeholders de la plantilla
    keys = [key for key in variations.keys() if '{' + key + '}' in base_query_template]

    # Generar todas las combinaciones posibles de los valores de las variaciones
    # Asegurarse de que el orden de las claves es consistente con el orden en las variations
    values = [variations[key] for key in keys]
    all_combinations = list(itertools.product(*values))

    generated_queries = []
    for combo in all_combinations:
        # Crear un diccionario para el formato, mapeando las claves a los valores de la combinación actual
        format_dict = dict(zip(keys, combo))
        generated_queries.append(base_query_template.format(**format_dict))

    return generated_queries

In [19]:
#@title Motor de Priorización de Riesgo en Pruebas

print(f"\n--- Modificando `variations` y regenerando Casos de Prueba Combinatorios ---")

# Definir las variaciones extendidas para nuestros casos de prueba
# Incluyendo 'aniversario' y 'adulto' para activar la lógica de riesgo.
extended_variations = {
    "ocasion": ["cumpleaños", "navidad", "aniversario"],
    "genero": ["niño", "niña", "adulto"],
    "edad": [8, 10, 12, 30] # Añadir 30 para 'adulto de 30 años' como caso inusual
}

base_query_template = "regalo de {ocasion} para un {genero} de {edad} años"

# Regenerar los casos de prueba combinatorios con las variaciones extendidas
extended_combinatorial_queries = generate_combinatorial_test_cases(base_query_template, extended_variations)

print(f"Se generaron {len(extended_combinatorial_queries)} casos de prueba combinatorios con variaciones extendidas.")

# Asignar puntajes de riesgo a cada consulta regenerada
queries_with_extended_risk = []
for query in extended_combinatorial_queries:
    score = assign_risk_score(query)
    queries_with_extended_risk.append({"query": query, "risk_score": score})

# Ordenar las consultas por puntaje de riesgo de mayor a menor
sorted_extended_queries_by_risk = sorted(queries_with_extended_risk, key=lambda x: x["risk_score"], reverse=True)

print("\nCasos de Prueba Ordenados por Nivel de Riesgo (de mayor a menor) con variaciones extendidas:")
for i, item in enumerate(sorted_extended_queries_by_risk):
    print(f"{i+1}. Riesgo: {item['risk_score']}, Consulta: '{item['query']}'")

print("--------------------------------------------------")


--- Modificando `variations` y regenerando Casos de Prueba Combinatorios ---
Se generaron 36 casos de prueba combinatorios con variaciones extendidas.

Casos de Prueba Ordenados por Nivel de Riesgo (de mayor a menor) con variaciones extendidas:
1. Riesgo: 16, Consulta: 'regalo de aniversario para un niño de 30 años'
2. Riesgo: 16, Consulta: 'regalo de aniversario para un niña de 30 años'
3. Riesgo: 16, Consulta: 'regalo de aniversario para un adulto de 8 años'
4. Riesgo: 16, Consulta: 'regalo de aniversario para un adulto de 10 años'
5. Riesgo: 16, Consulta: 'regalo de aniversario para un adulto de 12 años'
6. Riesgo: 11, Consulta: 'regalo de cumpleaños para un niño de 30 años'
7. Riesgo: 11, Consulta: 'regalo de cumpleaños para un niña de 30 años'
8. Riesgo: 11, Consulta: 'regalo de cumpleaños para un adulto de 8 años'
9. Riesgo: 11, Consulta: 'regalo de cumpleaños para un adulto de 10 años'
10. Riesgo: 11, Consulta: 'regalo de cumpleaños para un adulto de 12 años'
11. Riesgo: 11, Co

In [20]:
#@title Verificación de datos: Comprueba si test_results existe y tiene datos


if 'test_results' not in globals() or not test_results:
    print("ADVERTENCIA: 'test_results' no está definida o está vacía. Usando datos de ejemplo para demostrar la estructura.\n")
    test_results = [
        {
            "query": "ejemplo de consulta para un regalo",
            "risk_score": 5,
            "execution_status": "success",
            "final_response": "Aquí tienes un artículo: Nombre: Regalo, Descripción: Un bonito regalo, img_url: http://ejemplo.com/img.jpg",
            "error_message": None,
            "execution_time_ms": 15000,
            "tool_calls": "(datos de llamada a herramienta de ejemplo)"
        }
    ]

print(
    "--- Esquema de Resultados de Pruebas ---\n\n"
    "La estructura de datos para almacenar los resultados de las pruebas (`test_results`) "
    "ya ha sido establecida y poblada.\n\n"
    "Aquí tienes un ejemplo de los primeros 5 resultados registrados:\n"
)

for i, result in enumerate(test_results[:5]):
    print(f"Resultado de Prueba {i+1}:")
    for key, value in result.items():
        print(f"  {key}: {value}")
    print("\n")

ADVERTENCIA: 'test_results' no está definida o está vacía. Usando datos de ejemplo para demostrar la estructura.

--- Esquema de Resultados de Pruebas ---

La estructura de datos para almacenar los resultados de las pruebas (`test_results`) ya ha sido establecida y poblada.

Aquí tienes un ejemplo de los primeros 5 resultados registrados:

Resultado de Prueba 1:
  query: ejemplo de consulta para un regalo
  risk_score: 5
  execution_status: success
  final_response: Aquí tienes un artículo: Nombre: Regalo, Descripción: Un bonito regalo, img_url: http://ejemplo.com/img.jpg
  error_message: None
  execution_time_ms: 15000
  tool_calls: (datos de llamada a herramienta de ejemplo)




### Aprenda de ejecuciones previas utilizando modelos de predicción.

Implementa bucle que itere sobre casos priorizados, llame a `test_agent`, imprima resultados Y APRENDA de ejecuciones previas usando modelos de predicción.

In [28]:
# Importar modelo de ML (ejemplo conceptual)
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import time # Import the time module

# Inicializar modelo para aprendizaje (si no existe)
if 'prediction_model' not in globals():
    prediction_model = RandomForestClassifier()
    print("Modelo de predicción inicializado para aprendizaje de ejecuciones previas")

# Inicializar test_results, historical_features y historical_labels
# Aseguramos que historical_features y historical_labels siempre estén definidas.
if 'test_results' not in globals():
    test_results = []
historical_features = []
historical_labels = []

print("--- Ejecutando pruebas con aprendizaje ---\n")

# Antes del bucle, asegurar que prioritized_test_cases existe
if 'prioritized_test_cases' not in globals():
    # Obtener casos del módulo de priorización
    prioritized_test_cases = sorted_extended_queries_by_risk  # Del módulo 2
    print(f"✓ {len(prioritized_test_cases)} casos de prueba cargados")

# Bucle de ejecución de pruebas
for i, test_case in enumerate(prioritized_test_cases[:20]):
    print(f"\n▶ Ejecutando caso {i+1}: {test_case}")

    # PREDICCIÓN basada en ejecuciones previas
    # Only attempt prediction if the model has been fitted (i.e., historical_features has data)
    if 'prediction_model' in globals() and len(historical_features) > 0:
        case_features = [
            test_case.get('risk_score', 0),
            len(test_case.get('query', '')),
        ]
        try:
            # Reshape case_features to 2D array if it's a single sample
            failure_probability = prediction_model.predict_proba(np.array(case_features).reshape(1, -1))[0][1]
            print(f"  Predicción de fallo basada en ejecuciones previas: {failure_probability:.2%}")
        except Exception as e:
            print(f"  Error en la predicción del modelo: {e}")

    # Ejecutar prueba (conceptual)
    import asyncio
    result_to_append = {}
    start_time = time.time() # Start time for measuring execution
    try:
        final_response_text = await test_agent(test_case['query'], shop_agent)
        end_time = time.time() # End time
        execution_duration_ms = int((end_time - start_time) * 1000)

        # Create a structured result for test_results
        result_to_append = {
            "query": test_case['query'],
            "risk_score": test_case['risk_score'],
            "execution_status": "success",
            "final_response": final_response_text,
            "error_message": None,
            "execution_time_ms": execution_duration_ms,
            "tool_calls": "(tool call details not captured by current test_agent)" # Placeholder or improve test_agent
        }
        print(f"  Estado: {result_to_append['execution_status']}")

    except Exception as e:
        end_time = time.time() # End time even on error
        execution_duration_ms = int((end_time - start_time) * 1000) # Ensure execution_duration_ms is defined
        error_msg = str(e)
        result_to_append = {
            "query": test_case['query'],
            "risk_score": test_case['risk_score'],
            "execution_status": "failed",
            "final_response": None,
            "error_message": error_msg,
            "execution_time_ms": execution_duration_ms,
            "tool_calls": "N/A (error during execution)"
        }
        print(f"  Estado: {result_to_append['execution_status']}")
        print(f"  Error: {error_msg}")
    print(f"  Tiempo: {result_to_append.get('execution_time_ms', 0)}ms")

    test_results.append(result_to_append)

    # Añadir a datos de entrenamiento para FUTURO aprendizaje
    features_for_learning = [
        result_to_append.get('risk_score', 0),
        len(result_to_append.get('query', '')),
        1 if result_to_append.get('execution_status') == 'success' else 0
    ]
    label_for_learning = 1 if result_to_append.get('execution_status') == 'failed' else 0 # Label 1 for failure, 0 for success

    historical_features.append(features_for_learning)
    historical_labels.append(label_for_learning)

# Entrenar modelo con todas las ejecuciones previas *después* del bucle
if len(historical_features) > 0:
    # Ensure features and labels are numpy arrays for scikit-learn
    prediction_model.fit(np.array(historical_features), np.array(historical_labels))
    print(f"\n✓ Modelo entrenado con {len(historical_features)} ejecuciones previas (después del bucle)")

print(f"""
  ✓ Total pruebas ejecutadas: {len(test_results)}
  ✓ Datos de aprendizaje acumulados: {len(historical_features)} ejemplos
  ✓ Modelo listo para mejorar predicciones en próximas ejecuciones""")

--- Ejecutando pruebas con aprendizaje ---


▶ Ejecutando caso 1: {'query': 'regalo de aniversario para un niño de 30 años', 'risk_score': 16}

>>> User Query: regalo de aniversario para un niño de 30 años
  Estado: failed
  Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 22.444342924s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violation